In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Economy,AtlasIntel,NaN,12/15/2025,12/19/2025,2315.0,A,2.050922,1.583896,NaN,35.0,60.0,-25.000000,34.62851,59.979620,-25.351110,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,17:00:50 31 Dec 2025
1,Donald Trump (2nd term),Immigration,AtlasIntel,NaN,12/15/2025,12/19/2025,2315.0,A,2.050922,1.583896,NaN,41.0,57.0,-16.000000,41.34027,55.337300,-13.997030,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,17:00:52 31 Dec 2025
2,Donald Trump (2nd term),Trade,AtlasIntel,NaN,12/15/2025,12/19/2025,2315.0,A,2.050922,1.583896,NaN,35.0,61.0,-26.000000,34.42171,61.092380,-26.670670,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,17:00:56 31 Dec 2025
3,Donald Trump (2nd term),Trade,YouGov,Economist,12/20/2025,12/22/2025,1592.0,A,1.294688,1.294688,NaN,36.0,56.0,-20.000000,37.05619,56.959800,-19.903610,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/22/2025,17:00:56 31 Dec 2025
4,Donald Trump (2nd term),Strong,Trafalgar Group,NaN,12/26/2025,12/27/2025,1098.0,LV,1.291294,1.217782,NaN,46.7,37.5,9.200001,40.02650,39.665310,0.361191,https://www.thetrafalgargroup.org/wp-content/u...,12/27/2025,17:00:42 31 Dec 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,Donald Trump (2nd term),Weak,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,15.0,8.0,7.000000,12.82960,6.759334,6.070266,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:46 31 Dec 2025
3531,Donald Trump (2nd term),Strong,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,29.0,29.0,0.000000,33.33362,29.063110,4.270508,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:42 31 Dec 2025
3532,Donald Trump (2nd term),Cost,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.996709,0.000000,NaN,35.0,41.0,-6.000000,38.42154,41.183780,-2.762241,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,17:00:54 31 Dec 2025
3533,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800.0,RV,1.093180,0.000000,NaN,56.0,39.0,17.000000,52.88565,42.432460,10.453190,https://insideradvantage.com/first-insideradva...,1/20/2025,17:00:36 31 Dec 2025


In [4]:
# approval[approval['pollster']=='YouGov'][approval['subgroup']=='All polls']

In [5]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'All polls'])]

In [6]:
approval.to_csv('trump_approval_polls_RV.csv')

In [7]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [8]:
approval.shape

(629, 20)

In [9]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [10]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [11]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [12]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [13]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [14]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [15]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [16]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [17]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [18]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(629, 20)

In [19]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,Trafalgar Group,NaN,12/26/2025,12/27/2025,1098.0,LV,1.291294,1.217782,...,44.6,5.600002,47.35702,47.03993,0.317093,https://www.thetrafalgargroup.org/wp-content/u...,12/27/2025,17:00:36 31 Dec 2025,Trafalgar Group,-0.215439
1,Donald Trump (2nd term),Voters,YouGov,Economist,12/26/2025,12/29/2025,1420.0,RV,0.942115,0.942115,...,55.0,-13.000000,42.58463,53.94235,-11.357720,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/29/2025,17:00:36 31 Dec 2025,YouGov,0.047635
2,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/19/2025,12/29/2025,1500.0,LV,0.905550,0.685486,...,54.0,-10.000000,41.27386,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/29/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015
3,Donald Trump (2nd term),Voters,J.L. Partners,Daily Mail,12/20/2025,12/21/2025,1000.0,RV,1.026039,0.606689,...,47.0,-4.000000,43.47617,50.44818,-6.972012,https://www.dailymail.co.uk/news/article-15420...,12/21/2025,17:00:36 31 Dec 2025,J.L. Partners,0.197911
4,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,12/19/2025,12/20/2025,800.0,LV,1.086069,0.590037,...,41.1,8.400002,46.38565,44.53246,1.853191,https://insideradvantage.com/national-trump-ap...,12/20/2025,17:00:36 31 Dec 2025,InsiderAdvantage,0.034562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.000000,51.15123,44.36165,6.789581,https://pro.morningconsult.com/trackers/donald...,1/26/2025,17:00:36 31 Dec 2025,Morning Consult,0.584994
625,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.000000,50.45238,42.79871,7.653671,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,17:00:36 31 Dec 2025,Echelon Insights,-0.216672
626,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.000000,49.27386,44.55045,4.723412,https://www.rasmussenreports.com/public_conten...,1/24/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015
627,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.000000,53.40107,41.66902,11.732050,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,17:00:36 31 Dec 2025,RMG Research,0.278023


In [20]:
approval.shape

(629, 22)

In [21]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     237
YouGov                                                 59
Morning Consult                                        49
RMG Research                                           46
J.L. Partners                                          18
Global Strategy Group/GBAO (Navigator Research)        18
Echelon Insights                                       14
Harris Insights & Analytics                            12
Emerson College                                        12
American Research Group                                11
Quinnipiac University                                  11
John Zogby Strategies                                   9
Clarity Campaign Labs                                   8
InsiderAdvantage                                        7
Beacon Research/Shaw & Co. Research                     7
Ipsos                                                   7
McLaughlin & Associates                                 6
SSRS 

In [22]:
# approval[~approval['tracking'].isna()]

In [23]:
approval['tracking'].value_counts()

tracking
True    237
Name: count, dtype: int64

In [24]:
# full_approval = approval.copy()

In [25]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,Trafalgar Group,NaN,12/26/2025,12/27/2025,1098.0,LV,1.291294,1.217782,...,44.6,5.600002,47.35702,47.03993,0.317093,https://www.thetrafalgargroup.org/wp-content/u...,12/27/2025,17:00:36 31 Dec 2025,Trafalgar Group,-0.215439
1,Donald Trump (2nd term),Voters,YouGov,Economist,12/26/2025,12/29/2025,1420.0,RV,0.942115,0.942115,...,55.0,-13.000000,42.58463,53.94235,-11.357720,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/29/2025,17:00:36 31 Dec 2025,YouGov,0.047635
2,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/19/2025,12/29/2025,1500.0,LV,0.905550,0.685486,...,54.0,-10.000000,41.27386,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/29/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015
3,Donald Trump (2nd term),Voters,J.L. Partners,Daily Mail,12/20/2025,12/21/2025,1000.0,RV,1.026039,0.606689,...,47.0,-4.000000,43.47617,50.44818,-6.972012,https://www.dailymail.co.uk/news/article-15420...,12/21/2025,17:00:36 31 Dec 2025,J.L. Partners,0.197911
4,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,12/19/2025,12/20/2025,800.0,LV,1.086069,0.590037,...,41.1,8.400002,46.38565,44.53246,1.853191,https://insideradvantage.com/national-trump-ap...,12/20/2025,17:00:36 31 Dec 2025,InsiderAdvantage,0.034562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.000000,51.15123,44.36165,6.789581,https://pro.morningconsult.com/trackers/donald...,1/26/2025,17:00:36 31 Dec 2025,Morning Consult,0.584994
625,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.000000,50.45238,42.79871,7.653671,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,17:00:36 31 Dec 2025,Echelon Insights,-0.216672
626,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.000000,49.27386,44.55045,4.723412,https://www.rasmussenreports.com/public_conten...,1/24/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015
627,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.000000,53.40107,41.66902,11.732050,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,17:00:36 31 Dec 2025,RMG Research,0.278023


In [26]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [27]:
# house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+\d.\d\d.*@@([\d\.\+\-]+)')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,-1.999072
1,D,1.017374
2,D,0.632896
3,D,0.791334
4,R,-0.597041
...,...,...
93,R,-2.443137
94,R,-0.446172
95,R,-3.234717
96,R,-4.939213


In [28]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[1]
house_eff

,0,1,house_effect
0,-1,-1.999072,-1.999072
1,1,1.017374,1.017374
2,1,0.632896,0.632896
3,1,0.791334,0.791334
4,-1,-0.597041,-0.597041
...,...,...,...
93,-1,-2.443137,-2.443137
94,-1,-0.446172,-0.446172
95,-1,-3.234717,-3.234717
96,-1,-4.939213,-4.939213


In [29]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-1.999072
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.017374
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.632896
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.791334
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.597041
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.443137
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.446172
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.234717


In [30]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [31]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
2,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/19/2025,12/29/2025,1500.0,LV,0.905550,0.685486,...,41.27386,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/29/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/12/2025,12/18/2025,1500.0,LV,0.338629,0.092337,...,42.27386,54.55045,-12.276590,https://www.rasmussenreports.com/public_conten...,12/18/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,44.361664,53.638336
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/18/2025,12/24/2025,1500.0,LV,0.122665,0.070231,...,41.27386,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/24/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/17/2025,12/23/2025,1500.0,LV,0.120555,0.061548,...,42.27386,55.55045,-13.276590,https://www.rasmussenreports.com/public_conten...,12/23/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,44.361664,54.638336
30,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/16/2025,12/22/2025,1500.0,LV,0.119004,0.053264,...,41.27386,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/22/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Donald Trump (2nd term),Voters,North Star Opinion Research,Defending Democracy Together,3/13/2025,3/17/2025,1000.0,RV,1.158492,0.000000,...,46.48283,50.51826,-4.035431,https://www.defendingdemocracytogether.org/app...,3/17/2025,17:00:36 31 Dec 2025,North Star Opinion Research,0.043339,46.000000,51.000000
223,Donald Trump (2nd term),Voters,American Pulse Research & Polling,NaN,6/23/2025,6/25/2025,633.0,RV,0.813438,0.000000,...,48.59026,49.63150,-1.041241,https://s3.amazonaws.com/kajabi-storefronts-pr...,6/25/2025,17:00:36 31 Dec 2025,American Pulse Research & Polling,0.800463,49.700000,48.900000
478,Donald Trump (2nd term),Voters,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,1.404286,0.000000,...,49.56556,48.56862,0.996941,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,17:00:36 31 Dec 2025,OnMessage Inc.,-0.600249,48.227044,50.772956
549,Donald Trump (2nd term),Voters,Yale Youth Poll,NaN,10/29/2025,11/11/2025,3426.0,RV,1.154554,0.000000,...,42.61671,55.02629,-12.409580,https://youthpoll.yale.edu/fall-2025-results,11/11/2025,17:00:36 31 Dec 2025,NaN,NaN,42.800000,55.200000


In [32]:
approval.shape

(628, 22)

In [33]:
approval = approval_adj.copy()
approval.shape

(628, 24)

In [34]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'] + approval['sponsors'].fillna('none'))[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
2,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/19/2025,12/29/2025,1500.0,LV,0.905550,0.685486,...,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/29/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336,0
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/12/2025,12/18/2025,1500.0,LV,0.338629,0.092337,...,54.55045,-12.276590,https://www.rasmussenreports.com/public_conten...,12/18/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,44.361664,53.638336,1
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/18/2025,12/24/2025,1500.0,LV,0.122665,0.070231,...,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/24/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336,2
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/17/2025,12/23/2025,1500.0,LV,0.120555,0.061548,...,55.55045,-13.276590,https://www.rasmussenreports.com/public_conten...,12/23/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,44.361664,54.638336,3
30,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/16/2025,12/22/2025,1500.0,LV,0.119004,0.053264,...,55.55045,-14.276590,https://www.rasmussenreports.com/public_conten...,12/22/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336,4
31,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/15/2025,12/19/2025,1500.0,LV,0.161912,0.052211,...,54.55045,-12.276590,https://www.rasmussenreports.com/public_conten...,12/19/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,44.361664,53.638336,5
33,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/5/2025,12/11/2025,1500.0,LV,0.331632,0.034474,...,52.55045,-8.276588,https://www.rasmussenreports.com/public_conten...,12/11/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,6


In [35]:
approval['pollster_id'] = pd.factorize(approval['pollster'] + approval['sponsors'].fillna('none'))[0]

In [36]:
approval['population'] = approval['population'].str.strip(' ')

In [37]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['RV'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['LV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [38]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [39]:
approval['population'].value_counts().index.values

array(['RV', 'LV', 'V'], dtype=object)

In [40]:
approval.shape, pipeline(approval).shape

((628, 26), (624, 26))

In [41]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
2,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/19/2025,2025-12-29,1500.0,LV,0.905550,0.685486,...,-14.276590,https://www.rasmussenreports.com/public_conten...,12/29/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336,0,0
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/12/2025,2025-12-18,1500.0,LV,0.338629,0.092337,...,-12.276590,https://www.rasmussenreports.com/public_conten...,12/18/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,44.361664,53.638336,1,0
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/18/2025,2025-12-24,1500.0,LV,0.122665,0.070231,...,-14.276590,https://www.rasmussenreports.com/public_conten...,12/24/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336,2,0
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/17/2025,2025-12-23,1500.0,LV,0.120555,0.061548,...,-13.276590,https://www.rasmussenreports.com/public_conten...,12/23/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,44.361664,54.638336,3,0
30,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/16/2025,2025-12-22,1500.0,LV,0.119004,0.053264,...,-14.276590,https://www.rasmussenreports.com/public_conten...,12/22/2025,17:00:36 31 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Donald Trump (2nd term),Voters,North Star Opinion Research,Defending Democracy Together,3/13/2025,2025-03-17,1000.0,RV,1.158492,0.000000,...,-4.035431,https://www.defendingdemocracytogether.org/app...,3/17/2025,17:00:36 31 Dec 2025,North Star Opinion Research,0.043339,46.000000,51.000000,621,69
223,Donald Trump (2nd term),Voters,American Pulse Research & Polling,NaN,6/23/2025,2025-06-25,633.0,RV,0.813438,0.000000,...,-1.041241,https://s3.amazonaws.com/kajabi-storefronts-pr...,6/25/2025,17:00:36 31 Dec 2025,American Pulse Research & Polling,0.800463,49.700000,48.900000,622,70
478,Donald Trump (2nd term),Voters,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2025-02-19,1000.0,LV,1.404286,0.000000,...,0.996941,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,17:00:36 31 Dec 2025,OnMessage Inc.,-0.600249,48.227044,50.772956,623,71
549,Donald Trump (2nd term),Voters,Yale Youth Poll,NaN,10/29/2025,2025-11-11,3426.0,RV,1.154554,0.000000,...,-12.409580,https://youthpoll.yale.edu/fall-2025-results,11/11/2025,17:00:36 31 Dec 2025,NaN,NaN,42.800000,55.200000,624,72


In [42]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [43]:
full_approval = approval.copy()

In [44]:
approval = approval[approval['tracking'].isna()]

In [45]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [46]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
1,Donald Trump (2nd term),Voters,YouGov,Economist,12/26/2025,12/29/2025,1420.0,RV,0.942115,0.942115,...,-11.357720,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/29/2025,17:00:36 31 Dec 2025,YouGov,0.047635,42.249587,54.750413,236,2
5,Donald Trump (2nd term),Voters,YouGov,Economist,12/20/2025,12/22/2025,1425.0,RV,0.816699,0.466609,...,-12.357720,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/22/2025,17:00:36 31 Dec 2025,YouGov,0.047635,42.249587,55.750413,237,2
15,Donald Trump (2nd term),Voters,YouGov,Economist,12/12/2025,12/15/2025,1453.0,RV,0.776830,0.188072,...,-10.357720,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/15/2025,17:00:36 31 Dec 2025,YouGov,0.047635,43.249587,54.750413,238,2
29,Donald Trump (2nd term),Voters,YouGov,Economist,12/5/2025,12/8/2025,1380.0,RV,0.734024,0.056053,...,-9.357719,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/8/2025,17:00:36 31 Dec 2025,YouGov,0.047635,43.249587,53.750413,239,2
38,Donald Trump (2nd term),Voters,YouGov,Economist,11/28/2025,12/1/2025,1456.0,RV,0.703955,0.016401,...,-12.357720,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,17:00:36 31 Dec 2025,YouGov,0.047635,42.249587,55.750413,240,2


In [47]:
approval[approval['pollster']=='Civiqs']# [['approve', 'disapprove', 'approve_adj', 'disapprove_adj']]

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
287,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,5/17/2025,5/20/2025,1018.0,RV,1.120966,0.0,...,-4.326431,https://civiqs.com/documents/Civiqs_DailyKos_b...,5/20/2025,17:00:36 31 Dec 2025,Civiqs,0.158562,47.860928,51.139072,572,44
363,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,4/12/2025,4/15/2025,1124.0,RV,1.149724,0.0,...,-6.326431,https://civiqs.com/reports/2025/04/17/report-a...,4/15/2025,17:00:36 31 Dec 2025,Civiqs,0.158562,46.860928,52.139072,573,44
396,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,2/28/2025,3/3/2025,1031.0,RV,1.124632,0.0,...,-3.326431,https://civiqs.com/reports/2025/03/04/report-a...,3/3/2025,17:00:36 31 Dec 2025,Civiqs,0.158562,48.860928,51.139072,574,44


In [48]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id', 'pollster_id'], dtype=object)

In [49]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id', 'pollster_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,pollster_id
1,YouGov,Economist,12/26/2025,12/29/2025,1420.0,RV,42.0,55.0,42.249587,54.750413,-13.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,236,2
5,YouGov,Economist,12/20/2025,12/22/2025,1425.0,RV,42.0,56.0,42.249587,55.750413,-14.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,237,2
15,YouGov,Economist,12/12/2025,12/15/2025,1453.0,RV,43.0,55.0,43.249587,54.750413,-12.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,238,2
29,YouGov,Economist,12/5/2025,12/8/2025,1380.0,RV,43.0,54.0,43.249587,53.750413,-11.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,239,2
38,YouGov,Economist,11/28/2025,12/1/2025,1456.0,RV,42.0,56.0,42.249587,55.750413,-14.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,240,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,North Star Opinion Research,Defending Democracy Together,3/13/2025,3/17/2025,1000.0,RV,46.0,51.0,46.000000,51.000000,-5.000000,https://www.defendingdemocracytogether.org/app...,0.043339,621,69
223,American Pulse Research & Polling,NaN,6/23/2025,6/25/2025,633.0,RV,49.7,48.9,49.700000,48.900000,0.799999,https://s3.amazonaws.com/kajabi-storefronts-pr...,0.800463,622,70
478,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,50.0,49.0,48.227044,50.772956,1.000000,https://onmessageinc.com/wp-content/uploads/20...,-0.600249,623,71
549,Yale Youth Poll,NaN,10/29/2025,11/11/2025,3426.0,RV,42.8,55.2,42.800000,55.200000,-12.400000,https://youthpoll.yale.edu/fall-2025-results,NaN,624,72


In [50]:
approval_polls.shape

(379, 15)

In [51]:
def zone_flood_checker(data, pid, date):
    """
    Check whether there are multiple polls from the same pollster + same sponsor (if applicable) within the last two weeks, 
    which could indicate "flooding the zone." Return number of polls from same pollster/sponsor in last two weeks.

    :param data: Polls dataframe.
    :param pid: Pollster ID, specified in the polls dataframe as the column pollster_id.
    :param date: Date at which specified poll is conducted. Corresponds to column end_date in polls dataframe.
    """
    thres = date - pd.Timedelta(weeks=2)
    df = data[ (data['pollster_id'] == pid) & (pd.to_datetime(data['end_date']) >= thres) ]
    return max(df.shape[0], 1)

In [52]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(30))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual): # For pollsters without a quality rating from the Silver Bulletin
            return 0.2
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

   # Multiple polls in short window (aka "flooding the zone")
    def zone_flood_weight(pid, date):
        num_polls = zone_flood_checker(approval_polls, pid, date)
        return 1 / np.sqrt(num_polls)
    state_pivot['zone_flood_weight'] = state_pivot[['pollster_id', 'end_date']].apply(
        lambda x: zone_flood_weight(x['pollster_id'], x['end_date']),
        axis=1
    )

    # Gather the weights together
    state_pivot['total_weights'] = (state_pivot['sample_size_weights'] * state_pivot['time_weights'] * 
                                    state_pivot['quality_weights'] * state_pivot['population_weights'] * state_pivot['zone_flood_weight'])
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [53]:
approval_polls['sample_size'].isna().any()

np.False_

In [54]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
1,YouGov,Economist,12/26/2025,2025-12-29,1420.0,RV,42.0,55.0,42.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,236,2,0.002560,8.576959e-01,0.964021,0.8,0.577350,9.043979e-02
5,YouGov,Economist,12/20/2025,2025-12-22,1425.0,RV,42.0,56.0,42.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,237,2,0.002565,5.011872e-01,0.964021,0.8,0.500000,4.584797e-02
15,YouGov,Economist,12/12/2025,2025-12-15,1453.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,238,2,0.002590,2.928645e-01,0.964021,0.8,0.447214,2.419676e-02
29,YouGov,Economist,12/5/2025,2025-12-08,1380.0,RV,43.0,54.0,43.249587,53.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,239,2,0.002524,1.711328e-01,0.964021,0.8,0.408248,1.257882e-02
38,YouGov,Economist,11/28/2025,2025-12-01,1456.0,RV,42.0,56.0,42.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,240,2,0.002593,1.000000e-01,0.964021,0.8,0.377964,6.989954e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,North Star Opinion Research,Defending Democracy Together,3/13/2025,2025-03-17,1000.0,RV,46.0,51.0,46.000000,51.000000,...,https://www.defendingdemocracytogether.org/app...,0.043339,621,69,0.002149,2.326305e-10,0.967212,0.8,1.000000,3.577211e-11
223,American Pulse Research & Polling,NaN,6/23/2025,2025-06-25,633.0,RV,49.7,48.9,49.700000,48.900000,...,https://s3.amazonaws.com/kajabi-storefronts-pr...,0.800463,622,70,0.001710,5.011872e-07,0.200000,0.8,1.000000,1.267909e-08
478,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2025-02-19,1000.0,LV,50.0,49.0,48.227044,50.772956,...,https://onmessageinc.com/wp-content/uploads/20...,-0.600249,623,71,0.002149,3.162278e-11,1.586817,1.0,1.000000,9.972249e-12
549,Yale Youth Poll,NaN,10/29/2025,2025-11-11,3426.0,RV,42.8,55.2,42.800000,55.200000,...,https://youthpoll.yale.edu/fall-2025-results,NaN,624,72,0.003722,2.154435e-02,0.200000,0.8,1.000000,1.186534e-03


In [55]:
avgs_pipeline(approval_polls, datetime.datetime.today()).shape

(377, 21)

In [56]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [57]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,56.212570,39.228331
1,2025-01-24,53.663752,41.072584
2,2025-01-25,53.663752,41.072584
3,2025-01-26,53.593626,41.214518
4,2025-01-27,50.834890,42.261068
...,...,...,...
338,2025-12-27,43.830241,52.165859
339,2025-12-28,43.830241,52.165859
340,2025-12-29,43.687287,52.399605
341,2025-12-30,43.687287,52.399605


In [58]:
polls_df.shape

(377, 21)

In [59]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
1,YouGov,Economist,12/26/2025,2025-12-29,1420.0,RV,42.0,55.0,42.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,236,2,0.002560,0.857696,0.964021,0.8,0.577350,0.090440
5,YouGov,Economist,12/20/2025,2025-12-22,1425.0,RV,42.0,56.0,42.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,237,2,0.002565,0.501187,0.964021,0.8,0.500000,0.045848
15,YouGov,Economist,12/12/2025,2025-12-15,1453.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,238,2,0.002590,0.292864,0.964021,0.8,0.447214,0.024197


In [60]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [61]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-12-26,2025-12-29,1420 RV,9.043979e-02,42.0,55.0,-13.000000
1,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-12-20,2025-12-22,1425 RV,4.584797e-02,42.0,56.0,-14.000000
2,"<a href=""https://pro.morningconsult.com/tracke...",2025-12-19,2025-12-21,2203 RV,1.264735e-02,45.0,52.0,-7.000000
3,"<a href=""https://www.dailymail.co.uk/news/arti...",2025-12-20,2025-12-21,1000 RV,6.337329e-02,43.0,47.0,-4.000000
4,"<a href=""https://insideradvantage.com/national...",2025-12-19,2025-12-20,800 LV,7.440435e-02,49.5,41.1,8.400002
...,...,...,...,...,...,...,...,...
372,"<a href=""https://poll.qu.edu/poll-release?rele...",2025-01-23,2025-01-27,1019 RV,3.274335e-13,46.0,43.0,3.000000
373,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,3.489942e-14,52.0,44.0,8.000000
374,"<a href=""https://echeloninsights.com/in-the-ne...",2025-01-22,2025-01-24,1024 LV,2.832449e-13,51.0,43.0,8.000000
375,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,1.305143e-13,57.0,39.0,18.000000


In [62]:
display_df.to_csv('polls_display_table_RV.csv')

In [63]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,0.758850,0.262355
1,2025-01-24,2.661521,1.894681
2,2025-01-25,2.661521,1.894681
3,2025-01-26,2.600537,1.928781
4,2025-01-27,3.815328,2.050034
...,...,...,...
338,2025-12-27,3.200423,4.373884
339,2025-12-28,3.200423,4.373884
340,2025-12-29,3.085755,4.236763
341,2025-12-30,3.085755,4.236763


In [64]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,56.212570,39.228331,16.984238
1,2025-01-24,53.663752,41.072584,12.591169
2,2025-01-25,53.663752,41.072584,12.591169
3,2025-01-26,53.593626,41.214518,12.379108
4,2025-01-27,50.834890,42.261068,8.573823
...,...,...,...,...
338,2025-12-27,43.830241,52.165859,-8.335617
339,2025-12-28,43.830241,52.165859,-8.335617
340,2025-12-29,43.687287,52.399605,-8.712318
341,2025-12-30,43.687287,52.399605,-8.712318


In [65]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,56.212570,39.228331,57.699915,54.725224,39.742546,38.714116
1,2025-01-24,53.663752,41.072584,58.880333,48.447171,44.786159,37.359009
2,2025-01-25,53.663752,41.072584,58.880333,48.447171,44.786159,37.359009
3,2025-01-26,53.593626,41.214518,58.690677,48.496574,44.994929,37.434106
4,2025-01-27,50.834890,42.261068,58.312934,43.356846,46.279134,38.243001
...,...,...,...,...,...,...,...
338,2025-12-27,43.830241,52.165859,50.103069,37.557413,60.738671,43.593046
339,2025-12-28,43.830241,52.165859,50.103069,37.557413,60.738671,43.593046
340,2025-12-29,43.687287,52.399605,49.735366,37.639208,60.703661,44.095549
341,2025-12-30,43.687287,52.399605,49.735366,37.639208,60.703661,44.095549


In [66]:
# Export approval ratings
approval_trendline.to_csv('approval_stats_RV.csv')

In [67]:
polls_df.shape

(377, 22)

In [68]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [69]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [70]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [71]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [72]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,16.984238,1.021204
1,2025-01-24,12.591169,4.549973
2,2025-01-25,12.591169,4.549973
3,2025-01-26,12.379108,4.502473
4,2025-01-27,8.573823,5.473213
...,...,...,...
338,2025-12-27,-8.335617,6.797561
339,2025-12-28,-8.335617,6.797561
340,2025-12-29,-8.712318,6.592043
341,2025-12-30,-8.712318,6.592043


In [73]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,16.984238,1.021204,18.985799,14.982678
1,2025-01-24,12.591169,4.549973,21.509115,3.673222
2,2025-01-25,12.591169,4.549973,21.509115,3.673222
3,2025-01-26,12.379108,4.502473,21.203955,3.554261
4,2025-01-27,8.573823,5.473213,19.301320,-2.153675
...,...,...,...,...,...
338,2025-12-27,-8.335617,6.797561,4.987603,-21.658838
339,2025-12-28,-8.335617,6.797561,4.987603,-21.658838
340,2025-12-29,-8.712318,6.592043,4.208087,-21.632723
341,2025-12-30,-8.712318,6.592043,4.208087,-21.632723


In [74]:
net_approval.to_csv('net_approval_RV.csv')

### Auxiliary Graph: Individual Pollsters

In [75]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id',
       'pollster_id'], dtype=object)

In [76]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
appr = appr[~appr['pred_plus_minus'].isna()]
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'J.L. Partners', 'Global Strategy Group/GBAO (Navigator Research)',
       'Echelon Insights', 'Emerson College',
       'Harris Insights & Analytics', 'American Research Group',
       'Quinnipiac University', 'John Zogby Strategies',
       'Clarity Campaign Labs', 'Ipsos',
       'Beacon Research/Shaw & Co. Research', 'InsiderAdvantage', 'SSRS',
       'McLaughlin & Associates', 'Marquette University Law School',
       'Marist College', 'TIPP Insights', 'Targoz Market Research',
       'GBAO', 'Fabrizio Lee & Associates/Impact Research',
       'Trafalgar Group', 'Civiqs', 'Wick', 'Cygnal', 'co/efficient',
       'Hart Research Associates/Public Opinion Strategies',
       'Blueprint Polling', 'OH Predictive Insights',
       'The New York Times/Siena College', 'NewsNation',
       'High Point University', 'The Winston Group',
       'Normington, Petts & Associates', 'Public Opinion Strategies',
       

In [77]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [78]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [79]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [80]:
appr.shape

(597, 27)

In [81]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [82]:
get_ppm('YouGov')

np.float64(0.047635015)

In [83]:
indiv_pollster_avg('YouGov')

(np.float64(42.32930680275095),
 np.float64(55.183413758468966),
 np.float64(0.5645181223097119),
 np.float64(0.6111492511650902),
 np.float64(-12.854106955718029),
 np.float64(1.036249141907317))

In [84]:
indiv_pollster_avg('RMG Research')

(np.float64(48.19805231418897),
 np.float64(50.33333633998013),
 np.float64(1.946404050098188),
 np.float64(1.544225209435078),
 np.float64(-2.135284025791162),
 np.float64(3.472591842656574))

In [85]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [86]:
# nets

In [87]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
9,American Research Group,-26.977330,0.309123,0.818141,11
16,SSRS,-21.998431,0.125238,0.966835,6
34,High Point University,-18.999130,0.088491,0.411680,2
13,Ipsos,-16.610781,2.006344,0.442898,7
19,Marist College,-15.203043,0.640681,-0.436610,6
5,Global Strategy Group/GBAO (Navigator Research),-14.202817,0.984215,0.804988,18
10,Quinnipiac University,-14.002256,0.072813,-0.290484,11
33,NewsNation,-13.000000,0.000656,0.614459,2
22,GBAO,-12.999847,0.024751,0.384788,3
1,YouGov,-12.854107,1.036249,0.047635,59


In [88]:
# indiv_avgs.to_csv('indiv_pollster_avgs_RV.csv')

In [89]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-2.85842042]), np.float64(-7.457080353512424))

In [90]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.21628005])